In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Import data
import numpy as np
import pandas as pd
np.random.seed(4705)

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.columns

Index(['post_text', 'topic', 'author', 'label', 'id', 'word_count',
       'words_pronom', 'words_per_sen', 'words_over_6', 'pos_emo', 'neg_emo',
       'count_noun', 'count_verb', 'count_adj'],
      dtype='object')

In [0]:
# Select topic and shuffle data
topic = "abortion"
selected = data[data.topic==topic]
selected = np.random.permutation(selected)
#set(selected[:,1])

In [0]:
# Import functions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV

# 2. Method 1 - Ngrams (no idf)

In [0]:
# Show that ngram_range don't need to be params because of feature selection 
# cv = CountVectorizer(stop_words='english', ngram_range=(1,4))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(3,4))
# print(np.shape(cv.fit_transform(selected[:,0])))

(1708, 237620)
(1708, 74275)
(1708, 163345)


In [0]:
np.shape(y)

(1708,)

In [0]:
# Set label as y
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected[:,0])

In [0]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6077309598532009
clf__alpha: 1
exf__k: 500


In [0]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6135789130695752
clf__alpha: 0.5
exf__k: 50


In [0]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6141877175833033
clf__alpha: 1
exf__k: 100


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [0]:
print(gs.best_score_, gs.best_params_)

0.6071375898201026 {'clf__C': 100, 'clf__gamma': 0.001, 'clf__kernel': 'rbf', 'exf__k': 100}


## 找到abortion最好组合(Ngram)：0.614
0.6141877175833033
clf__alpha: 1
exf__k: 100

ComplementNB


## Gay rights

In [0]:
# Select topic and shuffle data
topic2 = "gay rights"
selected2 = data[data.topic==topic2]
selected2 = np.random.permutation(selected2)
#set(selected2[:,1])
# Set label as y
y = selected2[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected2[:,0])

In [0]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6210507819363908
clf__alpha: 1
exf__k: 20


In [0]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6166119243610018
clf__alpha: 1
exf__k: 1000


In [0]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5791249341064839
clf__alpha: 1
exf__k: 500


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [0]:
print(gs.best_score_, gs.best_params_)

0.6453401457091494 {'clf__C': 100, 'clf__gamma': 0.0001, 'clf__kernel': 'rbf', 'exf__k': 20}


## 找到gay rights最好组合（Ngram）:0.645, 'clf__C': 100, 'clf__gamma': 0.0001, 'clf__kernel': 'rbf', 'exf__k': 20, SVC

# 3. Method 2-Other features (with idf?)

## Abortion,先不用idf

In [0]:
set(selected[:,1])

{'abortion'}

In [0]:
# Extract features
# Select topic and shuffle data
# Set label as y
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected[:,0])
#idf???????

from scipy.sparse import hstack
X_more = hstack((X, selected[:,5:].astype(float)))

In [0]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5842842688343537
clf__alpha: 1
exf__k: 1000


In [0]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6065596542676339
clf__alpha: 1
exf__k: 1000


In [0]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5690761605871962
clf__alpha: 0.1
exf__k: 500


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)

KeyboardInterrupt: ignored

In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC(kernel='rbf')),
])
params = {'exf__k':[20,50,100,500],
          'clf__C': [0.1,1, 10, 100],
          'clf__gamma': [1e-3, 1e-4],
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)

In [0]:
print(gs.best_score_, gs.best_params_)

0.6007031263397986 {'clf__C': 100, 'clf__gamma': 0.0001, 'exf__k': 500}


## 试试SGDClassifier->loss=hinge是SVM

In [0]:
from sklearn.linear_model import SGDClassifier
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6106292123270052
clf__alpha: 0.01
clf__max_iter: 5000
clf__penalty: 'None'
exf__k: 1000


In [0]:

#2->5,4->7,6->9
X_more = hstack((X, selected[:,7:9].astype(float)))
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier()),
])
parameters_SVM = {
    'exf__k':[20,50,100,500,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__loss':('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'clf__max_iter':(1000,5000,10000,20000),
    
}
gs_clf_SVM = GridSearchCV(SGD, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

KeyboardInterrupt: ignored

## 对other feature, svc换别的参数看看1.+idf, 2.只考虑6个liwc

In [0]:
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,3))
X = X.fit_transform(selected[:,0])

from scipy.sparse import hstack
X_more = hstack((X, selected[:,5:8].astype(float)))

from sklearn.linear_model import SGDClassifier
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,800,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6153127197269812
clf__alpha: 0.001
clf__max_iter: 1000
clf__penalty: 'l1'
exf__k: 800


In [0]:
X_more = hstack((X, selected[:,5:11].astype(float)))

SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,800,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.612405892541716
clf__alpha: 0.001
clf__max_iter: 1000
clf__penalty: 'l1'
exf__k: 2000


#其实加了后面几个feature没啥用

In [0]:
np.shape(selected[:,5:11])

(1708, 6)

# Advanced Pipeline for Method2

In [0]:
np.shape(selected)

(1708, 14)

In [0]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import FeatureUnion
class AddLiwc(TransformerMixin):
  def transform(self, X, **transform_params):
    return DictVectorizer(X[:, 5:11])
  
  def fit(self, X, y=None, **fit_params):
    return self
  
class GetText(TransformerMixin):
  def transform(self, X, **transform_params):
    return X[:, 0]
  
  def fit(self, X, y=None, **fit_params):
    return self  

In [0]:
pipe = Pipeline([
    ('features', FeatureUnion([
       ('ngram_tf_idf',Pipeline([
           ('txt', GetText()),
           ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))),
           #('tfidf', TfidfTransformer())
       ])), 
       ('liwc', AddLiwc())
    ])),
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB())    
])
params = {
  #'tfidf__use_idf': [False],
  'exf__k':[20],
  'clf__alpha':[1],
#   'tfidf__use_idf': (True, False),
#   'exf__k':(20,50,100,500,1000,2000),
#   'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}

In [0]:
from sklearn.feature_extraction import DictVectorizer
cv = CountVectorizer(stop_words='english', ngram_range=(1,3))
t1 = cv.fit_transform(selected[:,0])
t2 = DictVectorizer(selected[:,-1])

In [0]:
FeatureUnion([])

TypeError: ignored

In [0]:
pipe = Pipeline([
    ('features', FeatureUnion([
       ('ngram_tf_idf',Pipeline([
           ('txt', GetText()),
           ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3)))
       ])), 
       ('liwc', AddLiwc())
    ])),
    ('clf', MultinomialNB())    
])
pipe.fit(selected[:,:-1], selected[:,3])

ValueError: ignored

In [0]:
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(selected[:,:-1], selected[:,3])
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

TypeError: ignored

In [0]:
pipe = Pipeline([
  ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))),
  ('exf', SelectKBest(chi2)),
  ('clf', MultinomialNB()),
])
params = {
  'exf__k':(20,50,100,500,1000,2000),
  'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(selected[:,0], selected[:,3])
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

In [0]:
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

# My search
每一组参数，都得到平均的CV结果，然后再从平均值中选最大的。

所以可以是search调用run

run中分train 和test.

In [16]:
params_NB = {
    'exf_k':(20,50,100,500,1000,2000),
    'clf_alpha':(1,0.5,1e-1,1e-2,1e-3),
    
}
comb_NB = []
for i in range(len(params_NB['exf_k'])):
  for j in range(len(params_NB['clf_alpha'])):
    comb_NB.append((params_NB['exf_k'][i], params_NB['clf_alpha'][j]))
print(comb_NB[:6])

[(20, 1), (20, 0.5), (20, 0.1), (20, 0.01), (20, 0.001), (50, 1)]


In [28]:
params_SVM = {
    'exf_k':[50,100,500,1000],
		'clf_loss':['hinge','squared_hinge'],
		'clf_C': [1, 10, 50,100],
		'clf_max_iter':[1000,2000,3000],
		'clf_class_weight':[None,'balanced']
}
comb_SVM = []
for i in params_SVM['exf_k']:
  for j in params_SVM['clf_loss']:
    for l in params_SVM['clf_C']:
      for m in params_SVM['clf_max_iter']:
        for t in params_SVM['clf_class_weight']:
          comb_SVM.append((i,j,l,m,t))
print(comb_SVM[:6])

[(50, 'hinge', 1, 1000, None), (50, 'hinge', 1, 1000, 'balanced'), (50, 'hinge', 1, 2000, None), (50, 'hinge', 1, 2000, 'balanced'), (50, 'hinge', 1, 3000, None), (50, 'hinge', 1, 3000, 'balanced')]


In [0]:
from scipy.sparse import hstack

In [0]:
def run(selected, paras, method=1, NB=True, search=False):
  kfolds = 5
  test_size = len(selected)//kfolds
  score,f1 = 0,0
  
  cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
  exf_k = paras[0]
  kBest = SelectKBest(chi2, k=exf_k)
  if NB:
    clf = MultinomialNB(alpha=paras[1])
  else:
    clf = LinearSVC(loss=paras[1],C=paras[2],max_iter=paras[3],class_weight=paras[4])
  
  for i in range(kfolds):
    test = selected[i*test_size:(i+1)*test_size]
    train = np.vstack((selected[:i*test_size,:],selected[(i+1)*test_size:,:]))
    
    trainY = train[:,3]
    testY = test[:,3]
    trainX = cv.fit_transform(train[:,0])
    testX = cv.transform(test[:,0])
    if method == 2:
      #tfidf = TfidfTransformer()
      #trainX = tfidf.fit_transform(trainX)
      trainX = hstack((trainX, train[:,5:11].astype(float)))
      #testX = tfidf.transform(testX)
      testX = hstack((testX, test[:,5:11].astype(float)))
    trainX = kBest.fit_transform(trainX, trainY)
    testX = kBest.transform(testX)
    
    clf.fit(trainX, trainY)
    pred = clf.predict(testX)
    score += metrics.accuracy_score(testY, pred)
    #average in f1 could be any of 'macro','micro','weighted'
    f1 += metrics.f1_score(testY, pred, average='micro')
  if not search:
    print("The averaged accuracy score of 5-fold cv is %0.2f and f1 score is %0.2f." %(score/5, f1/5))
  else:
    return score/5


In [0]:
def run(selected, paras, method=1, NB=True, search=False):
  kfolds = 5
  test_size = len(selected)//kfolds
  score,f1 = 0,0
  
  cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
  exf_k = paras[0]
  kBest = SelectKBest(chi2, k=exf_k)
  if NB:
    clf = MultinomialNB(alpha=paras[1])
  else:
    clf = LinearSVC(loss=paras[1],C=paras[2],max_iter=paras[3],class_weight=paras[4])
  
  for i in range(kfolds):
    test = selected[i*test_size:(i+1)*test_size]
    train = np.vstack((selected[:i*test_size,:],selected[(i+1)*test_size:,:]))
    
    trainY = train[:,3]
    testY = test[:,3]
    trainX = cv.fit_transform(train[:,0])
    testX = cv.transform(test[:,0])
    if method == 2:
      #tfidf = TfidfTransformer()
      #trainX = tfidf.fit_transform(trainX)
      trainX = hstack((trainX, train[:,5:11].astype(float)))
      #testX = tfidf.transform(testX)
      testX = hstack((testX, test[:,5:11].astype(float)))
    trainX = kBest.fit_transform(trainX, trainY)
    testX = kBest.transform(testX)
    
    clf.fit(trainX, trainY)
    pred = clf.predict(testX)
    score += metrics.accuracy_score(testY, pred)
    #average in f1 could be any of 'macro','micro','weighted'
    f1 += metrics.f1_score(testY, pred, average='micro')
  if not search:
    print("The averaged accuracy score of 5-fold cv is %0.2f and f1 score is %0.2f." %(score/5, f1/5))
  else:
    return score/5


In [70]:
topic = "abortion"
selected = data[data.topic==topic]
#selected = np.random.permutation(selected)

run(selected,(3000,1e-10),method=1,NB=True,search=False)

The averaged accuracy score of 5-fold cv is 0.53 and f1 score is 0.53.


In [0]:
from sklearn.model_selection import KFold
def run2(selected,paras,NB=True,method=1):
  kf = KFold(n_splits=5, shuffle= True, random_state=13)
  cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
  exf_k = paras[0]
  score, f1 = 0,0
  kBest = SelectKBest(chi2, k=exf_k)
  if NB:
    clf = MultinomialNB(alpha=paras[1])
  else:
    clf = LinearSVC(loss=paras[1],C=paras[2],max_iter=paras[3],class_weight=paras[4])
  # run each cross validation
  for train_index, test_index in kf.split(selected):
    test = selected[test_index]
    train =selected[train_index]
    
    trainY = train[:,3]
    testY = test[:,3]
    trainX = cv.fit_transform(train[:,0])
    testX = cv.transform(test[:,0])
    if method == 2:
      #tfidf = TfidfTransformer()
      #trainX = tfidf.fit_transform(trainX)
      trainX = hstack((trainX, train[:,5:11].astype(float)))
      #testX = tfidf.transform(testX)
      testX = hstack((testX, test[:,5:11].astype(float)))
    trainX = kBest.fit_transform(trainX, trainY)
    testX = kBest.transform(testX)
    
    clf.fit(trainX, trainY)
    pred = clf.predict(testX)
    score += metrics.accuracy_score(testY, pred)
    #average in f1 could be any of 'macro','micro','weighted'
    f1 += metrics.f1_score(testY, pred, average='micro')  
  print("The averaged accuracy score of 5-fold cv is %0.2f and f1 score is %0.2f." %(score/5, f1/5))

In [74]:
run2(selected, (3000,1e-10))

The averaged accuracy score of 5-fold cv is 0.53 and f1 score is 0.53.


In [14]:
run(selected, (20,1),method=2)

The averaged accuracy score of 5-fold cv is 0.50 and f1 score is 0.50.


In [0]:
def MySearch(selected, NB=True, method=2):
  best_score = 0
  if NB:
    comb_paras = comb_NB
  else:
    comb_paras = comb_SVM
  for ipar in comb_paras:
    tmp = run(selected, ipar, method=method,NB=NB,search=True)
    print("cross validation with paras:", ipar, ", score:", tmp)
    if tmp > best_score:
      best_score = tmp
      best_paras = ipar
  return best_score, best_paras

In [25]:
print(MySearch(selected))

cross validation with paras: (20, 1) , score: 0.5489736070381231
cross validation with paras: (20, 0.5) , score: 0.5489736070381231
cross validation with paras: (20, 0.1) , score: 0.5483870967741935
cross validation with paras: (20, 0.01) , score: 0.5483870967741935
cross validation with paras: (20, 0.001) , score: 0.5483870967741935
cross validation with paras: (50, 1) , score: 0.5524926686217009
cross validation with paras: (50, 0.5) , score: 0.5524926686217009
cross validation with paras: (50, 0.1) , score: 0.5542521994134897
cross validation with paras: (50, 0.01) , score: 0.55366568914956
cross validation with paras: (50, 0.001) , score: 0.5530791788856304
cross validation with paras: (100, 1) , score: 0.5577712609970674
cross validation with paras: (100, 0.5) , score: 0.5565982404692081
cross validation with paras: (100, 0.1) , score: 0.5560117302052785
cross validation with paras: (100, 0.01) , score: 0.5577712609970674
cross validation with paras: (100, 0.001) , score: 0.557771

In [27]:
print(MySearch(selected,method=1))

cross validation with paras: (20, 1) , score: 0.5870967741935484
cross validation with paras: (20, 0.5) , score: 0.5865102639296188
cross validation with paras: (20, 0.1) , score: 0.5853372434017595
cross validation with paras: (20, 0.01) , score: 0.5841642228739004
cross validation with paras: (20, 0.001) , score: 0.5841642228739004
cross validation with paras: (50, 1) , score: 0.6005865102639296
cross validation with paras: (50, 0.5) , score: 0.6017595307917889
cross validation with paras: (50, 0.1) , score: 0.6023460410557184
cross validation with paras: (50, 0.01) , score: 0.6023460410557184
cross validation with paras: (50, 0.001) , score: 0.6023460410557184
cross validation with paras: (100, 1) , score: 0.5976539589442815
cross validation with paras: (100, 0.5) , score: 0.5970674486803519
cross validation with paras: (100, 0.1) , score: 0.5988269794721408
cross validation with paras: (100, 0.01) , score: 0.5982404692082112
cross validation with paras: (100, 0.001) , score: 0.5982

In [0]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [0]:
print(MySearch(selected, NB=False, method=2))

NB=False, method=2最好的是
(0.6093841642228739, (500, 'squared_hinge', 1, 2000, 'balanced')

In [39]:
run(selected, (500, 'squared_hinge', 1, 2000, 'balanced'), method=2, NB=False, search=False)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


The averaged accuracy score of 5-fold cv is 0.58 and f1 score is 0.58.


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
run(selected, (500, 'squared_hinge', 1, 3000, None), method=2, NB=False, search=False)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


The averaged accuracy score of 5-fold cv is 0.58 and f1 score is 0.58.


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [40]:
topic = "gay rights"
selected = data[data.topic==topic]
selected = np.random.permutation(selected)
print(MySearch(selected, NB=False, method=2))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (50, 'hinge', 1, 1000, None) , score: 0.5933579335793358


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (50, 'hinge', 1, 1000, 'balanced') , score: 0.45239852398523983


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (1000, 'squared_hinge', 100, 3000, None) , score: 0.5734317343173432


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (1000, 'squared_hinge', 100, 3000, 'balanced') , score: 0.5512915129151292
(0.63690036900369, (50, 'hinge', 100, 3000, None))


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [48]:
run(selected, (50, 'hinge', 10, 1000, 'balanced'), method=2, NB=False, search=False)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


The averaged accuracy score of 5-fold cv is 0.50 and f1 score is 0.50.


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [49]:
topic = "abortion"
selected = data[data.topic==topic]
selected = np.random.permutation(selected)

params_SVM = {
    'exf_k':[50,250,500,750,2000],
		'clf_loss':['hinge','squared_hinge'],
		'clf_C': [0.5,1, 10, 20, 50,75],
		'clf_max_iter':[1000,2000,3000],
		'clf_class_weight':[None,'balanced']
}
comb_SVM = []
for i in params_SVM['exf_k']:
  for j in params_SVM['clf_loss']:
    for l in params_SVM['clf_C']:
      for m in params_SVM['clf_max_iter']:
        for t in params_SVM['clf_class_weight']:
          comb_SVM.append((i,j,l,m,t))
print(comb_SVM[:6])

print(MySearch(selected, NB=False, method=1))

[(50, 'hinge', 0.5, 1000, None), (50, 'hinge', 0.5, 1000, 'balanced'), (50, 'hinge', 0.5, 2000, None), (50, 'hinge', 0.5, 2000, 'balanced'), (50, 'hinge', 0.5, 3000, None), (50, 'hinge', 0.5, 3000, 'balanced')]


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (50, 'hinge', 0.5, 1000, None) , score: 0.6


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (50, 'hinge', 0.5, 1000, 'balanced') , score: 0.5835777126099706


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (2000, 'squared_hinge', 75, 3000, None) , score: 0.543108504398827


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


cross validation with paras: (2000, 'squared_hinge', 75, 3000, 'balanced') , score: 0.5472140762463342
(0.6152492668621701, (500, 'squared_hinge', 1, 3000, None))


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
run()